# 股票交易数据分析

In [1]:
%pwd

u'/Users/kamidox/work/stock-data'

In [3]:
import pandas as pd
import numpy as np

## 原始数据: 2000 年 - 2009 年 5 分钟历史成交数据

需要确保原始数据放在 raw 目录下，且每个年份单独一个目录。搜索 ghancn 可以免费下载 2009 年之前的数据。2009 年之后的需要购买。

In [5]:
d = pd.read_csv('raw/2008/SH600300.csv', names=['date','time','opening_price', 'ceiling_price', 'floor_price', 'closing_price', 'volume', 'amount'], header=None)

In [6]:
d.head()

,date,time,opening_price,ceiling_price,floor_price,closing_price,volume,amount
0,2008/01/02,09:35,9.89,9.90,9.76,9.77,3223.00,3174676
1,2008/01/02,09:40,9.77,9.84,9.75,9.79,2037.50,1992715
2,2008/01/02,09:45,9.78,9.82,9.78,9.79,1227.99,1203187
3,2008/01/02,09:50,9.80,9.85,9.79,9.85,1137.00,1115308
4,2008/01/02,09:55,9.84,9.95,9.84,9.94,1422.97,1405492


### 转化为日交易数据

In [7]:
g = d.groupby('date')

In [8]:
day = g.agg({'opening_price': 'first', 'ceiling_price': 'max', 'floor_price': 'min', 'closing_price': 'last', 'volume': 'sum', 'amount': 'sum'})

In [9]:
day.head()

,floor_price,opening_price,ceiling_price,volume,amount,closing_price
date,,,,,,
2008/01/02,9.75,9.89,10.31,87545.17,88759296,10.22
2008/01/03,10.22,10.31,10.88,151147.35,159856480,10.58
2008/01/04,10.28,10.58,10.76,81476.55,85745624,10.58
2008/01/07,10.45,10.48,11.09,130950.71,141612400,10.98
2008/01/08,10.30,10.95,11.00,116279.70,124193984,10.42


### 合并数据

可以参阅 stock.py 里的 `main()` 函数。把所有的数据转化为日交易数据，然后以股票代号为文件名保存在 `data` 目录下。

## 选股

什么股票是好股票？要回答这个问题，先要把最简单的问题说清楚。炒股就是低买高卖，实现获利。那么好股票的标准就是在你的习惯持股周期内，**波动最大的股票**。这很好理解吧，波动最大，我们才有可能在相对低点买入，在相对高点卖出，获利最大。

在一定的时间周期内，**衡量股票波动的指标定义为 最高价/最低价**。以我们表格中的数据，就是 ceiling_price/floor_price。这个比率最大的股票就是好股票。

关于时间周期，这个和个人的炒股习惯有关。有些人习惯做短线，可能就持股几天，或一两周。有些人习惯做长线，可能持股时长以年计算。这个完全是个人喜好问题。

有了这个思路，我们就可以玩转已经转换为日交易数据的股票，选出近期波动最大的股票。假设我们的目标是**选出一个月内波动最大的股票**。我们看一下如何用 pandas 实现这个目标。

### 过滤数据

我们先要按照考查周期来过滤数据。为了简单起见，我们假设一个月是22个交易日，且只计算交易时间，不计算停牌时间。

In [11]:
len(day)

239